<a href="https://colab.research.google.com/github/yuehaoshi/Movie_Recommendation_System/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
import re
from matplotlib import pyplot as plt
from sklearn.impute import SimpleImputer
from collections import Counter

#Data Cleaning and Peocessing

In [ ]:
data_ori = pd.read_csv("https://raw.githubusercontent.com/Noam-91/Movie_Recommendation_system/main/netflix_titles.csv")

In [ ]:
data_ori.head()
data_ori.info()

In [ ]:
data_ori.isnull().sum()

##Director, Cast and Country

In [ ]:
tv_total = data_ori[data_ori['type']=='TV Show'].shape[0]
missing_dir_tv = pd.isna(data_ori[data_ori['type']=='TV Show']['director']).sum()
movie_total = data_ori[data_ori['type']=='Movie'].shape[0]
missing_dir_movie = pd.isna(data_ori[data_ori['type']=='Movie']['director']).sum()
class1 = [missing_dir_tv, missing_dir_movie]
class2 = [tv_total-missing_dir_tv, movie_total-missing_dir_movie]

plt.bar(range(2), class1, width=.5)
plt.bar(range(2), class2, bottom=class1, width=.5)
plt.xticks(range(2), ('TV Shows', 'Movies'))
plt.legend(('Record without director ', 'Record with director'))
plt.show()

In [ ]:
data_movie = data_ori[data_ori['type'] == 'Movie']
data_tv = data_ori[data_ori['type'] == 'TV Show']
data_movie.drop(columns='type',axis=1,inplace=True)
data_tv.drop(columns='type',axis=1,inplace=True)

"prepare_country" function: take a dataFrame, firstly add the possible countries with its corresponding director (if have any), then drop rows with 'director', 'cast' or 'country' to be null

In [ ]:
def prepare_country(dataFrame):
    #Most movies have director and cast, so drop rows with empty director and cast first
    #Since most of TV shows do not have director, only drop empty cast for TV show
    dataFrame = dataFrame.dropna(subset = ['cast'])
    #Create a dictioary containing each director and their most frequent corresponding country, d_c_pair
    directors = dataFrame['director'].str.split(', ')
    directors1 = directors.dropna()
    director_set = set()
    for row in directors1:
        if pd.notnull(row[0]):
            for director in row:
                if director not in director_set:
                    director_set.add(director)  
    d_c = Counter()
    for idx, row in dataFrame.iterrows():
        if pd.notna(row['director']) and pd.notna(row['country']):
            director = row['director'].split(', ')
            country = row['country'].split(', ')
            for d in director:
                for c in country:
                    d_c[(d, c)] += 1
    d_c_pair = {}
    for i in director_set:
        max = 0
        max_c = None
        for j in d_c:
            d = j[0]
            c = j[1]
            if d == i:
                if d_c[j] >= max:
                    max = d_c[j]
                    max_c = c
        if max_c:
            d_c_pair[i] = max_c
    data = dataFrame.copy()
    data['director'] = data['director'].fillna('nan')
    data['country'] = data['country'].fillna(np.nan)
    data['director'] = data['director'].str.split(', ')
    condition = (pd.isna(data['country'])) & ('nan' not in data['director'])
    #If a row has director but no country, put the director's corresponding country (if exists) into the row
    for row in range(data.shape[0]):
        country_column = list(data.columns).index('country')
        director_column = list(data.columns).index('director')
        if (pd.isna(data.iloc[row, country_column])) & (data.iloc[row, director_column][0] != 'nan') & (data.iloc[row, director_column][0] in d_c_pair):
            data.iloc[row, country_column] = d_c_pair[data.iloc[row, director_column][0]] 
    #Delete the rows without country
    data = data.dropna(subset = ['country'])
    return data

In [ ]:
data_movie = data_movie.dropna(subset = ['director'])
data_movie = prepare_country(data_movie)
data_tv = prepare_country(data_tv)
print(data_movie.shape[0])
print(data_tv.shape[0])
plt.pie([data_movie.shape[0],data_tv.shape[0]], labels=['Movie',"TV Shows"])
plt.show()

In [ ]:
def one_hot(dataframe,prefix):
    if prefix != "director":
        classes = dataframe[prefix].str.split(', ')
    else:
        classes = dataframe[prefix]
    class_set = set()
    for row in classes:
        #print(row)
        for cls in row:
            if cls not in class_set:
                class_set.add(cls)
    df = pd.DataFrame()
    for cls in class_set:
        df[str(prefix+"_"+cls)] = dataframe[prefix].apply(lambda x: 1 if x.__contains__(cls) else 0)
    dataframe = pd.concat([dataframe,df], axis=1)
    dataframe.drop(columns=[prefix],axis=1, inplace=True)
    return dataframe

Compare to directly deleting all rows containing null in 'director', 'cast' or 'country', we have more rows kept in our modified database, which is because of our added country information inferred from the director's corresponding frequent country

In [ ]:
data_movie.isna().sum()

In [ ]:
data_movie

In [ ]:
data_movie = one_hot(data_movie, 'director')
data_movie = one_hot(data_movie, 'country')
data_movie = one_hot(data_movie, 'cast')
data_tv = one_hot(data_tv, 'cast') 
data_tv = one_hot(data_tv, 'country')

In [ ]:
data_movie.info()

In [ ]:
data_tv.info()

##Rating

In [ ]:
rating_series = data_ori['rating']
print(rating_series.describe())
print()
rating_series.value_counts()

There are three mistakes in the record. The 'duration' was mistakenly written into 'rating'. So we need to switch the value between these two columns. After that, we also do One-Hot encoding for 'rating'.

In [ ]:
def prepare_rating(dataset):
    row_idx = np.where(pd.isna(dataset['duration']))[0]
    if row_idx.size == 0: return dataset
    dur_idx = list(dataset.columns).index('duration')
    rat_idx = list(dataset.columns).index('rating')
    dataset.iloc[row_idx,dur_idx] = dataset.iloc[row_idx,rat_idx]
    dataset.iloc[row_idx,rat_idx] = np.nan
    data_rating = pd.get_dummies(dataset['rating'], drop_first=True)
    dataset = pd.concat([data_rating,dataset],axis=1)
    dataset.dropna(subset=['rating'])
    dataset.drop(columns=['rating'],axis=1, inplace=True)
    return dataset

In [ ]:
data_movie = prepare_rating(data_movie)
data_tv = prepare_rating(data_tv)

##Duration

Since duration contains two types of values: xxx min and xxx Seasons, we need to split them and put them into bins, because a classification makes more sense than regression in this task

In [ ]:
movie_dur_col = data_movie['duration'].apply(lambda x: x.split(" min")[0] if x.__contains__("min") else 0)
seasons_col = data_tv['duration'].apply(lambda x: x.split(" Season")[0] if x.__contains__("Season") else 0)
seasons_col = seasons_col.astype(int)
movie_dur_col = movie_dur_col.astype(int)

In [ ]:
seasons_col[(0<seasons_col)].hist(bins=np.arange(17), grid=False)

In [ ]:
movie_dur_col = movie_dur_col.astype(int)
movie_dur_col[(0<movie_dur_col)].hist(bins=[0,30,60,90,120,150,200,312], grid=False)

In [ ]:
def prepare_duration(dataset, is_movie):
    if is_movie:
        movie_dur_col = dataset['duration'].apply(lambda x: x.split(" min")[0] if x.__contains__("min") else 0)
        movie_dur_col = movie_dur_col.astype(int)
        df_movie_duration = pd.DataFrame()
        bins = [1,30,60,90,120,150,200,312]
        for x in range(len(bins)-1):
            col_name = "movie_duration_["+str(bins[x])+":"+str(bins[x+1])+"]"
            df_movie_duration[col_name] = movie_dur_col.apply(lambda y: 1 if bins[x]<=y and bins[x+1]>y else 0)
        dataset.drop(columns=['duration'],axis=1,inplace=True)
        return pd.concat([dataset,df_movie_duration],axis=1)
    else:
        seasons_col = dataset['duration'].apply(lambda x: x.split(" Season")[0] if x.__contains__("Season") else 0)
        seasons_col = seasons_col.astype(int)
        df_seasons = pd.DataFrame()
        bins = [1,3,17]
        for x in range(len(bins)-1):
            col_name = "seasons_["+str(bins[x])+":"+str(bins[x+1])+"]"
            df_seasons[col_name] = seasons_col.apply(lambda y: 1 if bins[x]<=y and bins[x+1]>y else 0)
        dataset.drop(columns=['duration'],axis=1,inplace=True)
        return pd.concat([dataset,df_seasons],axis=1)

In [ ]:
data_seasons = prepare_duration(data_tv,False)
data_movie_duration = prepare_duration(data_movie,True)

##Genres

A movie may belongs to multiple genre which is stacked in one column named as 'listed_in', we need to do one-hot encoding first.

In [ ]:
genres = data_ori['listed_in'].str.split(', ')
genres_set = set()
for row in genres:
    for genre in row:
        if genre not in genres_set:
            genres_set.add(genre)
data = data_ori.copy()
for genre in genres_set:
    data[genre] = data['listed_in'].apply(lambda x: 1 if x.__contains__(genre) else 0)
data = data.drop(columns=['listed_in'],axis=1)
data.columns[11:]

Since the 'type' feature has defined the object as TV shows or Movies, we need to combine some of the genres to reduce correlation.
'Docuseries' + 'Documentaries' --> 'Documentaries'
'TV Mysteries' --> 'Mysteries'
'TV Horror'+'Horror Movies' --> 'Horror'
'Thrillers'+'TV Thrillers' --> 'Thrillers'
'Spanish-Language TV Shows' --> 'Spanish'
'LGBTQ Movies' --> 'LGBTQ'
'Stand-Up Comedy & Talk Shows' + 'Stand-Up Comedy' --> 'Stand-Up Comedy'
'TV Action & Adventure' + 'Action & Adventure' --> Action & Adventure
'TV Shows' -- deleted
'Science & Nature TV'
'Crime TV Shows' --> crime
'British TV Shows' --> british
'Reality TV' --> reality
'Classic Movies' --> classic
'Sports Movies' --> sports
'Comedies' -- Keep
'Romantic Movies'+'Romantic TV Shows' --> Romantic
'International Movies'+'International TV Shows' --> international
'Dramas'+'TV Dramas' --> Dramas
'TV Sci-Fi & Fantasy'+'Sci-Fi & Fantasy' --> sci-fi & Fantasy
'Cult Movies'+ Classic & Cult TV --> cult
'Children & Family Movies'--> 'Children & Family'
'Music & Musicals'-- Keep
'Korean TV Shows' --> Korean
'Kids' TV' --> Kids
'Movies'-- deleted
'Anime Series'+'Anime Features' --> Anime
'Teen TV Shows' --> Teen

In [ ]:
def prepare_genre(dataset):
    data_copy = dataset.copy()
    genre_series = data_copy['listed_in']
    genres = genre_series.str.split(', ')
    genres_set = set()
    for row in genres:
        for genre in row:
            if genre not in genres_set:
                genres_set.add(genre)
    df = pd.DataFrame()
    for genre in genres_set:
      if genre=="Movies" or genre=="TV Shows":
        continue
      df[genre] = genre_series.apply(lambda x: 1 if x.__contains__(genre) else 0)
    data_copy.drop(columns=['listed_in'],axis=1,inplace=True)
    return pd.concat([data_copy,df],axis=1), df.columns

In [ ]:
data_movie, movie_g = prepare_genre(data_movie)
data_tv, tv_g = prepare_genre(data_tv)
movie_genre = data_movie[movie_g].sum(axis=0)
tv_genre = data_tv[tv_g].sum(axis=0)

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.pie(movie_genre.values, labels = movie_g)
ax2.pie(tv_genre.values, labels = tv_g)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
plt.pie(movie_genre.values, labels = movie_g)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
plt.pie(tv_genre.values, labels = tv_g)
plt.show()

For the 10 NaN found in date_added, we use the value one row above to fill it. Because the date_added is sorted in descending order. However, it shows that there are 14 entries which has data_added earlier than release_year, which is impossible. These are invalid inputs, so we decide to delete them.
From the user's point of view, the date the movie was added to the library is not important, so we decided not to adopt it.

In [ ]:
def prepare_release_add(dataset):
  dataset['date_added'].fillna(method='ffill',inplace=True)
  drop_row = dataset[dataset['date_added'].apply(lambda x: x.split(', ')[-1]).astype(int)<dataset['release_year']].index
  dataset.drop(drop_row,axis=0,inplace=True)
  dataset.drop(columns=['date_added'],axis=1,inplace=True)
  return dataset

In [ ]:
data_movie = prepare_release_add(data_movie)
data_tv = prepare_release_add(data_tv)